In [1]:
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 
import pyproj

In [2]:
#read all data 
cal_coords_list =[]
cal_y_list =[]
cal_X_list =[]
delt_stwr_intervel =[0.0]
csvFile = open("./Data_STWR/RealWorldData/precip_isotope_D3.csv", "r")
df = pd.read_csv(csvFile,header = 0,names=['Longitude','Latitude','Elevation','ppt','tmean','d2h','timestamp'],
                 dtype = {"Longitude" : "float64","Latitude":"float64",
                          "Elevation":"float64","ppt":"float64","tmean":"float64","d2h":"float64",
                          "timestamp":"float64"},
                 skip_blank_lines = True,
                 keep_default_na = False)

df.info()
df = df.sort_values(by=['timestamp'])  
all_data = df.values
tick_time = all_data[0,-1]
cal_coord_tick = []
cal_X_tick =[]
cal_y_tick =[]

time_tol = 1.0e-7

lensdata = len(all_data)
for row in range(lensdata):
    cur_time = all_data[row,-1]
    if(abs(cur_time-tick_time)>time_tol):
        cal_coords_list.append(np.asarray(cal_coord_tick))
        cal_X_list.append(np.asarray(cal_X_tick))
        cal_y_list.append(np.asarray(cal_y_tick))
        delt_t = cur_time - tick_time
        delt_stwr_intervel.append(delt_t) 
        tick_time =cur_time
        cal_coord_tick = []
        cal_X_tick =[]
        cal_y_tick =[]
    coords_tick = np.array([all_data[row,0],all_data[row,1]])
    cal_coord_tick.append(coords_tick)

    x_tick = np.array([all_data[row,2],all_data[row,3],all_data[row,4]])
    cal_X_tick.append(x_tick)
    y_tick = np.array([all_data[row,5]])
    cal_y_tick.append(y_tick)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 7 columns):
Longitude    272 non-null float64
Latitude     272 non-null float64
Elevation    272 non-null float64
ppt          272 non-null float64
tmean        272 non-null float64
d2h          272 non-null float64
timestamp    272 non-null float64
dtypes: float64(7)
memory usage: 15.0 KB


In [3]:
f = open('./Data_STWR/RealWorldData/output/testresults_d2h_73_LOOCV.txt','w')
error_stwr = []
error_gwr =[]
errsum_stwr = []
errsum_gwr=[]
errtol_stwr =[]
errtol_gwr =[]
spl_parts_cv = 73
cal_coord_last_tick= np.array_split(cal_coord_tick,spl_parts_cv,axis =0)
cal_last_X_tick = np.array_split(cal_X_tick, spl_parts_cv, axis = 0)
cal_last_y_tick = np.array_split(cal_y_tick, spl_parts_cv, axis = 0)
for i in range(spl_parts_cv):
        cal_x_gwr_cur = []
        cal_y_gwr_cur = []
        cal_coord_gwr_cur=[]
        cal_stwr_coordlist = cal_coords_list.copy()
        cal_stwr_Xlist =cal_X_list.copy()
        cal_stwr_ylist = cal_y_list.copy() 
        for j in range(spl_parts_cv):
                if (i != j):
                    # 作为训练集
                     cal_coord_gwr_cur.extend(cal_coord_last_tick[j])
                     cal_x_gwr_cur.extend(cal_last_X_tick[j])
                     cal_y_gwr_cur.extend(cal_last_y_tick[j])
        cal_X_gwr =np.asarray(cal_x_gwr_cur)
        cal_y_gwr = np.asarray(cal_y_gwr_cur)
        cal_cord_gwr = np.asarray(cal_coord_gwr_cur)
        cal_stwr_coordlist.append(cal_cord_gwr)
        cal_stwr_Xlist.append(cal_X_gwr)
        cal_stwr_ylist.append(cal_y_gwr)     
        #stwr 
        stwr_selector_ = Sel_Spt_BW(cal_stwr_coordlist, cal_stwr_ylist, cal_stwr_Xlist,#gwr_bw0,
                                    delt_stwr_intervel,spherical = True)
        optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search() 
        stwr_model = STWR(cal_stwr_coordlist,cal_stwr_ylist,cal_stwr_Xlist,delt_stwr_intervel,
                          optsita,opt_gwr_bw0,tick_nums=opt_btticks+1,alpha =optalpha,spherical = True,recorded = True)       
        stwr_results = stwr_model.fit()  
        f.write(stwr_results.summary())            
        stwr_scale = stwr_results.scale 
        stwr_residuals = stwr_results.resid_response     
        #gwr  
        gwr_selector = Sel_BW(cal_cord_gwr, cal_y_gwr, cal_X_gwr,spherical = True)
        gwr_bw= gwr_selector.search(bw_min=2)
        gwr_model = GWR(cal_cord_gwr, cal_y_gwr, cal_X_gwr, gwr_bw,spherical = True)
        gwr_results = gwr_model.fit()
        f.write(gwr_results.summary())
        gw_rscale = gwr_results.scale 
        gwr_residuals = gwr_results.resid_response
        #predition results
        predPointList = cal_coord_last_tick[i]
        PreX_list = cal_last_X_tick[i]
        vd_y = cal_last_y_tick[i]
        #stwr
        pred_stwr_dir_result = stwr_model.predict([predPointList],[PreX_list],stwr_scale,stwr_residuals)
        pre_y_stwr = pred_stwr_dir_result.predictions 
        #pre_parmas_stwr=np.reshape(pred_stwr_dir_result.params.flatten(),(-1,allklen_stwr)) 
        #gwr
        pred_gwr_dir_result = gwr_model.predict(predPointList,PreX_list,gw_rscale,gwr_residuals)
        pre_y_gwr = pred_gwr_dir_result.predictions
        #gwr
        #pre_parmas_gwr=np.reshape(pred_gwr_dir_result.params.flatten(),(-1,allklen_stwr))
        #error append
        lenpre = vd_y.shape[0]
        errsum_stwr.append(math.sqrt(np.sum(np.square(vd_y-pre_y_stwr))/lenpre))
        errsum_gwr.append(math.sqrt(np.sum(np.square(vd_y-pre_y_gwr))/lenpre))
        error_stwr.append(vd_y-pre_y_stwr)
        error_gwr.append(vd_y-pre_y_gwr)
        errtol_stwr.append(np.sum(np.square(vd_y-pre_y_stwr)))
        errtol_gwr.append(np.sum(np.square(vd_y-pre_y_gwr)))
        f.write('The result of {:d}'.format(i))
f.close()                  
sstw_total = np.sum(errsum_stwr)
sgwr_total = np.sum(errsum_gwr)
sstw_total
sgwr_total

Model type                                                         Gaussian
Number of observations:                                                 181
Number of covariates:                                                     4

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          52610.658
Log-likelihood:                                                    -339.548
AIC:                                                                687.096
AICc:                                                               690.005
BIC:                                                              52319.845
R2:                                                                   0.494
Adj. R2:                                                              0.472

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

1535.9976833732194